## WIP 3D SIM reconstruction template

This is a work in progress template for running 3D SIM reconstruction based on the Janelia Python and c SIM code written by David Hoffman and Lin Shao

## 1.  Define code paths
Currently we hard code these and they need to be modified to run on different machines.  In the future we may move to a more intelligent approach like always having code exist beside the notebooks and using relative imports.  

In [1]:
%pylab inline

import tifffile as tif
import os
import glob
%load_ext autoreload
%autoreload 
import shutil

# NOTE: BN the below code is from the legacy notebooks.  I am leaving it here for now, as it may be needed on some machines. 
if 'C:\\Users\\Cryo SIM-PALM\\Documents\\GitHub' in sys.path:
    sys.path.remove('C:\\Users\\Cryo SIM-PALM\\Documents\\GitHub')
else:
    pass

computer = 'default'

import sys

if computer == 'default':
    sys.path.insert(1, r'Y:\Cryo_data2\Data Processing Notebooks')
    sys.path.insert(1, r'C:\Users\Cryo SIM-PALM\code\simrecon\scripts')
    sys.path.insert(1, r'C:\Users\Cryo SIM-PALM\code\simrecon\scripts\Scripts')
elif computer == 'bnort':
    sys.path.insert(1, r'C:\Users\bnort\work\Janelia\code\\simrecon\scripts\Scripts')
    sys.path.insert(1, r'C:\Users\bnort\work\Janelia\code\\simrecon\scripts')
else:
    pass

import dphutils 
from simrecon_utils import simrecon, split_process_recombine

# import dask
import dask
from dask.diagnostics import ProgressBar

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


# Setup home directory and OTF directory:

Right now we leave in paths for the current test machines but in the future may move to a more intelligent approach (for example user chooses paths with dialog box, paths stored in configuration)

Home should start from {Hesslab(\\prfs.hhmi.org)} e.g. home = r'Y:\Cryo_data2\ORCA_data\3D SIM Cells'

#### Additonal legacy notes (BN: these notes were in the original notebook I got) I'm leaving them in for now in case we need them for troubleshhoting

There was an old note "OTF folder should be placed inside Data processing notebooks"

(BN I don't think this has to be the case, because there are many notebook that define the full OTF path)

OTF Folder directory should be e.g.:   Y:\Cryo_data2\Data Processing Notebooks\SIM PSFs OTFs

Raw data from V-SIM data acquisition should be placed inside a dated folder here:   'Y:\Cryo_data2\ORCA_data\3D SIM Cells' 

with data file directory structure e.g.: 'Y:\Cryo_data2\ORCA_data\3D SIM Cells\20240322\488 nm 5 phases 0.81 NA Linear SIM_cam1_0.mrc'

In [14]:
if computer == 'default': 
    home = r'F:\ORCA_Data\2024-12-03 - Schuyler LID 769\SIM imaging\LID 769 position 8 560nm - ROI 1 _20241204_105558'
    otf_path = r'Y:\Seyforth\Data For Brian\Brians OTF'#'Y:\Seyforth\Data For Brian\Brians OTF'
    #'Y:\Seyforth\Data For Brian\Cryo-SIM Scope #1 Data (Ground truth baseline)\Ground truth OTFs System #1'
elif computer == 'bnort':
    #home = r'D:\Janelia\Data 2024-06-06\Wiener, gammaApo and SupressR parameter testing\488nm comparison Brian'
    #home = r'D:\Janelia\Data 2024-10-02\560cm cell 4 _20240627_124604'
    home = r'D:\Janelia\Data 2024-10-10' 
    
    #otf_path = r'D:\Janelia\Data 2024-06-06\Wiener, gammaApo and SupressR parameter testing\OTF\BEAD 2 - NON-AR 1.2W 25ms retake_20240503_170242 BEST PSF!!\computed_OTF_folder'
    #otf_path = r'D:\Janelia\Data 2024-06-03\PSF-OTF used (Davids set of 4 wavelengths)\201909_19-20_best'
    #otf_path = r'C:\Users\bnort\work\Janelia\ims\OTF_folder'
OTFpath = os.path.join(otf_path,"*{}*.mrc")
OTFs = {wl : [path for path in glob.iglob(OTFpath.format(wl))] for wl in (560, 532, 488, 642)}
print(home)
OTFs

F:\ORCA_Data\2024-12-03 - Schuyler LID 769\SIM imaging\LID 769 position 8 560nm - ROI 1 _20241204_105558


{560: ['Y:\\Seyforth\\Data For Brian\\Brians OTF\\560 201909_19-20_best.mrc'],
 532: ['Y:\\Seyforth\\Data For Brian\\Brians OTF\\532 OTF Bead 3_20190920_154920.mrc'],
 488: ['Y:\\Seyforth\\Data For Brian\\Brians OTF\\488 nmLinOTF0_mask.mrc'],
 642: ['Y:\\Seyforth\\Data For Brian\\Brians OTF\\642 20240611_125236_best.mrc']}

## Set default params

Here we set the default params that will be used if none of them are overwritten.

Note that before calling the non-tiled and/or tiled processing code a small subset of parameters will be overwritten to values optimized for non-tiled/tiled processing

In [10]:
# David Solecki's suggestion: wiener=0.007 gammaApo=0.7 and suppressR=15
# original values by D.Hoffman: wiener=0.001 gammaApo=0.1 and suppressR=1.5

base_kwargs = dict(
                    nphases=5,
                    ndirs=3,
                    angle0= 1.29,
                    negDangle=True,              # James made False to try experiment
                    na= 0.85,
                    nimm= 1.0,
                    zoomfact= 2.0, 
                    background= 100.0,           # james experiment was 100.0
                    wiener= 0.007,
                    fastSIM=True,
                    otfRA= True,
                    dampenOrder0=True,
                    k0searchall=True,
                    equalizez=True,
                    preciseapo=True,
                    gammaApo=0.7,
                    suppressR=15.0
                )

def return_wl_otfs(path):
    if "488 Exc 532 Em" in path:
        wl = 532
    elif "488 Exc 642 Em" in path:
        wl = 642
    elif "532 Exc 561 Em" in path:
        wl = 561
    elif "560 nm" in path:
        wl = 560
    elif "488 nm" in path:
        wl = 488
    elif "532 nm" in path:
        wl = 532
    elif "642 nm" in path:
        wl = 642

    else:
        raise RuntimeError("no matching filename wavelength found, fix directory or filename or code")
    return wl, OTFs[wl]

## remove old processed data

This is commented out, but I assume we comment back in if we want to erase the previous run

In [11]:
'''python
# clear processed
for path in glob.glob(home + "/*/*proc *.mrc"):
    os.remove(path)
for path in glob.glob(home + "/*/*proc *.txt"):
    os.remove(path)
'''

'python\n# clear processed\nfor path in glob.glob(home + "/*/*proc *.mrc"):\n    os.remove(path)\nfor path in glob.glob(home + "/*/*proc *.txt"):\n    os.remove(path)\n'

# Define process function defined to use DASK and simrecon package

This function is annotated as a dask ```delayed``` function.  Which means it will not be called right away but put in a queue to call using dask. 

In [12]:
@dask.delayed
def process(sim_kwargs, output_file_name):
    sim_output = simrecon(**sim_kwargs)
    with open(output_file_name.replace(".mrc", ".txt"), "w") as myfile:
        myfile.write(str(sim_kwargs))
        myfile.write("\n".join(sim_output))
    return output_file_name

## Dictionary of wavelength based parameter sets

In [13]:
#parameter_sets_excitation_emission

#532nm excitation 650nm emission
dict_532nm_650nm = dict()
dict_532nm_650nm['wiener'] = .001
dict_532nm_650nm['forcemodamp'] = [0.8,0.25]
dict_532nm_650nm['otfamp'] = [1,1]
dict_532nm_650nm['nofilteroverlaps'] = True

#532nm excitation 650nm emission
dict_561nm_650nm = dict()
dict_561nm_650nm['wiener'] = .001
dict_561nm_650nm['forcemodamp'] = [0.8,0.25]
dict_561nm_650nm['otfamp'] = [1,1]
dict_561nm_650nm['nofilteroverlaps'] = True

#640nm excitation 650nm emission # default 642nm parameters
dict_642nm_650nm = dict()
dict_642nm_650nm['wiener'] = .0005
dict_642nm_650nm['forcemodamp'] = [0.8,0.25]
dict_642nm_650nm['otfamp'] = [1,1]
dict_642nm_650nm['nofilteroverlaps'] = True

#488nm excitation 515nm emission # default 488nm parameters
dict_488nm_515nm = dict()
dict_488nm_515nm['wiener'] = .0001
dict_488nm_515nm['forcemodamp'] = [0.8,0.15]
dict_488nm_515nm['otfamp'] = [1,1]
dict_488nm_515nm['nofilteroverlaps'] = True

#561nm excitation 615nm emission # default 561nm parameters
dict_488nm_515nm = dict()
dict_488nm_515nm['wiener'] = .00025
dict_488nm_515nm['forcemodamp'] = [0.8,0.25]
dict_488nm_515nm['otfamp'] = [1,1]
dict_488nm_515nm['nofilteroverlaps'] = True

## Set up parameters common between Non-tiled and tiled reconstruction

In [22]:
gammaApo = 0.3
suppressR = 15
wiener = 0.0001

user_text = 'gApo_'+str(gammaApo)+'_supR_'+str(suppressR)+'_w_'+str(wiener) 

nofilteroverlaps = False

keeporder2 = True


otf_amp = [1,1]
base_kwargs.update(dict(gammaApo = gammaApo, nofilteroverlaps=nofilteroverlaps, suppressR=suppressR, wiener=wiener, nthreads = 12))

if nofilteroverlaps==True:
    user_text += '_nofltroverlap'

    o1 = 1.0
    o2 = 0.055
    user_text += '_fmodamp_'+str(o1)+'_'+str(o2)
    forcemodamp = [o1, o2]
  
    base_kwargs.update(dict(forcemodamp=forcemodamp))   # default Full frame Recon. parameters 

    if 'keeporder2' in base_kwargs:
        del base_kwargs['keeporder2']
   
    
else:
    if 'forcemodamp' in  base_kwargs:
        del base_kwargs['forcemodamp']
        base_kwargs.update(dict(nofilteroverlaps=nofilteroverlaps))
    if keeporder2:
        user_text += '_keeporder2'
        base_kwargs.update(dict(keeporder2 = keeporder2))
       # del base_kwargs['otfamp']
    else:
        del base_kwargs['keeporder2']
 

print(base_kwargs)


{'nphases': 5, 'ndirs': 3, 'angle0': 1.29, 'negDangle': True, 'na': 0.85, 'nimm': 1.0, 'zoomfact': 2.0, 'background': 100.0, 'wiener': 0.0001, 'fastSIM': True, 'otfRA': True, 'dampenOrder0': True, 'k0searchall': True, 'equalizez': True, 'preciseapo': True, 'gammaApo': 0.3, 'suppressR': 15, 'nofilteroverlaps': False, 'nthreads': 12, 'keeporder2': True}


# Full frame Simrecon SIM image reconstruction, recon parameters: 

Set up full frame processing.  Note that we override gammaApo, supressR and wiener parameters.  Also note the code will not be run right away (just put in a dask queue)

In [98]:
to_process = []

done_already = set(glob.iglob(home + "/**/*proc*.mrc", recursive = True))
done_already;
done_already = set()     # do this if want to re-process
#print('Data processed already: \n')



for raw in glob.iglob(home + "/**/*SIM*.mrc", recursive = True):
   
    wl, otfs = return_wl_otfs(raw)
    for otf in otfs:
        if "proc" not in os.path.split(raw)[1]: 
            print('Data to process ' +str(raw))
            # Test with actually measured OTF
            sim_kwargs = dict(                                                                                                            
                input_file= raw,
                otf_file= otf,
                #ls= (wl/1000)/2/0.81,)
                ls = (wl/1000)/(2*0.775))
            
            sim_kwargs.update(base_kwargs)
            print(wl)
            OTF_filename = os.path.split(otf)[1].split('.')[0]
            print('\nOTF filename: ' +str(OTF_filename))
                
            #create processed file output name
            sim_kwargs["output_file"] = sim_kwargs["input_file"].replace(".mrc", '_proc' + OTF_filename + '_' 
                                                                         + user_text + ".mrc")
                
            if sim_kwargs["output_file"] not in done_already:
   
                print("\n\nSIM recon Data to save:" + '\n' + str(sim_kwargs["output_file"]) + '\n')
                to_process.append(process(sim_kwargs, sim_kwargs["output_file"]))

Data to process F:\ORCA_Data\11-22-2024 - Schuyler samples 2\LID768 SIM\LID 768 position 2 ROI 1 561nm_20241126_214319\560 nm 655 40 filter 5 phases 0.81 NA React_All Linear SIM_cam1_1.mrc
560

OTF filename: 560 201909_19-20_best


SIM recon Data to save:
F:\ORCA_Data\11-22-2024 - Schuyler samples 2\LID768 SIM\LID 768 position 2 ROI 1 561nm_20241126_214319\560 nm 655 40 filter 5 phases 0.81 NA React_All Linear SIM_cam1_1_proc560 201909_19-20_best_gApo_0.3_supR_15_w_0.0001_keeporder2.mrc



# Running cell below shows all files being processed for full frame reconstruction and progress using DASK

This cell will start and run the processing.  We use dask to compute our processing queue that was set up in the cell above.

In [99]:
with ProgressBar():
    print(to_process)
    out_names = done_already.update(*dask.compute(to_process))

[Delayed('process-c214afde-cd1d-456e-8b1c-eec506095a4d')]
[########################################] | 100% Completed | 123.70 s


# cell below checks for data already processed using the tiledreconstruction method, cell below that creates list of data to be set to done already, i.e. already processed


In [16]:
print(home)
print(glob.glob(home + "/**/*SIM_*.mrc"))

C:\1\2\gold_standard_set\532nm after all sequence and timing fixes_20240614_115706
[]


# Select general Simrecon tile parameters: # 

In [32]:
base_kwargs = dict(
                    nphases=5,
                    ndirs=3,
                    angle0= 1.36,
                    negDangle=True,              # James made False to try experiment
                    na= 0.85,
                    nimm= 1.0,
                    zoomfact= 2.0, 
                    background= 100.0,           # james experiment was 100.0
                    wiener= 0.007,
                    fastSIM=True,
                    otfRA= True,
                    dampenOrder0=True,
                    k0searchall=True,
                    equalizez=True,
                    preciseapo=True,
                    gammaApo=0.7,
                    suppressR=15.0
                )



In [7]:
for raw in glob.iglob(home + "/**/*SIM*.mrc", recursive = True):
    
    done_already = set(glob.iglob(home + "/**/*proc*.mrc", recursive = True))
    done_already;
    #done_already = set()     # do this if want to re-process

print("Datasets alread reconstructed: \n" + "\n".join(os.path.split(i)[0] for i in done_already))

Datasets alread reconstructed: 



# Select tile parmeters for tile reconstruction

In [14]:


# # # Tile size Parameters (units: pixels) # # #
tile_size = 64   #square tile size
tile_overlap = int(tile_size/2)        # square tile overlap [usually = tile_size/2]


# # # TILE Parameter settings # # # 
if tile_size == 64:
    # SIM RECON PARAMS
    gammaApo = 0.5
    suppressR = 2.0
    #force_modamp = [0.8,0.1]  # [0.8,0.1] for 488nm,[0.8,0.25] for 532nm, [0.8,0.4] for 561nm
elif tile_size == 32:
    # SIM RECON PARAMS
    gammaApo = 0.5
    suppressR = 2
elif tile_size == 16:
    # SIM RECON PARAMS
    gammaApo = 0.5
    suppressR = 0.5

user_text=''


# # # Constrained tile filter method # # #
filter_tiles = False
    
#user_text += user_text1 + user_text2 + user_text3 + user_text4 # comment out if your string/driectory path too long

 # # perform SIM recon wih No Filter overlaps # # #

nofilteroverlaps = False
keep_order2 = True

#update dataset processd, done_already list
done_already = []

for raw in glob.iglob(home + "/**/*SIM*.mrc", recursive = True):
    
    done_already = set(glob.iglob(home + "/**/*proc*.mrc", recursive = True))
    done_already;
    #done_already = set()     # do this if want to re-process

print("Datasets alread reconstructed: \n" + "\n".join(os.path.split(i)[0] for i in done_already))
    


for raw in glob.iglob(home + "/**/*SIM*.mrc", recursive = True):
    
    
    user_text_process = user_text
    wl, otfs = return_wl_otfs(raw)
    
    
    if filter_tiles:
        
    
        tile_limits = {}
   
        if wl == 488:
            tile_limits['spacing_min'] = 0.31   # magnitude vector inverse to line spacing value here; see simrecon text file output
            tile_limits['spacing_max'] = 0.32
            tile_limits['spacing_default']=0.361 # SIM angle
            tile_limits['angle_min'] = 1.2
            tile_limits['angle_max'] = 1.3
            tile_limits['angle_default'] = 1.29
            tile_limits['amp1_min'] = 0.0   # contrast of order 1
            tile_limits['amp1_max'] = 1.0
            tile_limits['amp1_default'] = 1.0
            tile_limits['amp2_min'] = 0.0  # contrast of order 2
            tile_limits['amp2_max'] = 1.0
            tile_limits['amp2_default'] = 1.0

        elif wl == 560:
            tile_limits['spacing_min'] = 0.355   # magnitude vector inverse to line spacing value here; see simrecon text file output
            tile_limits['spacing_max'] = 0.365
            tile_limits['spacing_default']=0.361 # SIM angle
            tile_limits['angle_min'] = 1.3
            tile_limits['angle_max'] = 1.4
            tile_limits['angle_default'] = 1.36
            tile_limits['amp1_min'] = 0.98   # contrast of order 1
            tile_limits['amp1_max'] = 1.02
            tile_limits['amp1_default'] = 1.0
            tile_limits['amp2_min'] = 0.98  # contrast of order 2
            tile_limits['amp2_max'] = 1.02
            tile_limits['amp2_default'] = 1.0

        elif wl == 642:
            tile_limits['spacing_min'] = 0.408  # magnitude vector inverse to line spacing value here; see simrecon text file output
            tile_limits['spacing_max'] = 0.416
            tile_limits['spacing_default']=0.412 # SIM angle
            tile_limits['angle_min'] = 1.2
            tile_limits['angle_max'] = 1.6
            tile_limits['angle_default'] = 1.36
            tile_limits['amp1_min'] = 0.98   # contrast of order 1
            tile_limits['amp1_max'] = 1.02
            tile_limits['amp1_default'] = 1.0
            tile_limits['amp2_min'] = 0.98  # contrast of order 2
            tile_limits['amp2_max'] = 1.02
            tile_limits['amp2_default'] = 1.0

        elif wl == 532:
            tile_limits['spacing_min'] = 0.336  # magnitude vector inverse to line spacing value here; see simrecon text file output
            tile_limits['spacing_max'] = 0.348
            tile_limits['spacing_default']=0.412 # SIM angle
            tile_limits['angle_min'] = 1.2
            tile_limits['angle_max'] = 1.6
            tile_limits['angle_default'] = 1.36
            tile_limits['amp1_min'] = 0.98   # contrast of order 1
            tile_limits['amp1_max'] = 1.02
            tile_limits['amp1_default'] = 1.0
            tile_limits['amp2_min'] = 0.98  # contrast of order 2
            tile_limits['amp2_max'] = 1.02
            tile_limits['amp2_default'] = 1.0
    
        user_text1 = 'mag_' + str(tile_limits['spacing_min'])  +"_" + str(tile_limits['spacing_max'])
        user_text2 = 'ang_' + str(tile_limits['angle_min'])  +"_" + str(tile_limits['angle_max'])
        user_text3 = 'A1_' + str(tile_limits['amp1_min'])  +"_" + str(tile_limits['amp1_max'])
        user_text4 = 'A2_' + str(tile_limits['amp2_min'])  +"_" + str(tile_limits['amp2_max'])
#         print('\n' + '\n' "TILING PARAMETERS: " + '_spacing_' + str(tile_limits['spacing_min'])  
#         +"_to_" + str(tile_limits['spacing_max'])+
#         '_angle_' + str(tile_limits['angle_min'])  +"_to_" + str(tile_limits['angle_max'])+
#         '_amp1_' + str(tile_limits['amp1_min'])  +"_to_" + str(tile_limits['amp1_max'])+
#         '_amp2_' + str(tile_limits['amp2_min'])  +"_to_" + str(tile_limits['amp2_max']))
        
    else:
        tile_limits = None
    
    if wl == 488:
        wiener = 0.001 # wiener = 0.0001 for 488nm, 0.00025 for 532nm,0.00025 for 561nm??? not sure??
        force_modamp = [1.0,0.055]  # [0.8,0.1] for 488nm,[0.8,0.25] for 532nm, [0.8,0.4] for 561nm
    elif wl == 532:
        wiener = 0.001 # wiener = 0.0001 for 488nm, 0.00025 for 532nm,0.00025 for 561nm??? not sure??
        force_modamp = [0.8,0.25]  # [0.8,0.1] for 488nm,[0.8,0.25] for 532nm, [0.8,0.4] for 561nm
    elif wl == 560:
        wiener = 0.0005 # wiener = 0.0001 for 488nm, 0.00025 for 532nm,0.00025 for 561nm??? not sure??
        force_modamp = [1.0,0.15]  # [0.8,0.1] for 488nm,[0.8,0.25] for 532nm, [0.8,0.4] for 561nm
    elif wl == 642:
        wiener = 0.0005 # wiener = 0.0001 for 488nm, 0.00025 for 532nm,0.00025 for 561nm??? not sure??
        force_modamp = [1.0,0.25]  # [0.8,0.1] for 488nm,[0.8,0.25] for 532nm, [0.8,0.4] for 561nm
    
    if nofilteroverlaps==True:
        user_text_process += '_NoFoverlaps_'
        user_text_process += 'Fmod_' + str(force_modamp)
        #user_text_process += 'forceotf_'
        if 'keeporder2' in  base_kwargs:
            del base_kwargs['keeporder2']
        base_kwargs.update(dict(nofilteroverlaps=nofilteroverlaps, forcemodamp = force_modamp))
    else:
        if 'forcemodamp' in  base_kwargs:
            del base_kwargs['forcemodamp']
        base_kwargs.update(dict(nofilteroverlaps=nofilteroverlaps))
        
    if keep_order2==True:
        user_text_process += '_keepod2_'
        #user_text_process += 'forceotf_'
        if 'otfamp' in  base_kwargs:
            del base_kwargs['otfamp']
        if 'nofilteroverlaps' in base_kwargs:
            del base_kwargs['nofilteroverlaps']
        base_kwargs.update(dict(keeporder2 = keep_order2))
        if 'forcemodamp' in  base_kwargs:
            del base_kwargs['forcemodamp']

    
    
  
    
  
    
    for otf in otfs:
                
        if "proc" not in os.path.split(raw)[1]: 
           
            print("\n" + "RAW DATA TO BE PROCESSED, PROCESS No.{}: ".format(step) + str(raw)+'\n')

            ls = (wl/1000)/(2*0.775)

            # Test with actually measured OTF

            sim_kwargs = dict(
                input_file= raw,
                otf_file= otf,
                ls= ls,  # Lambda/2*Na, units: microns
                #ls = 0.315
                
            )
            
            
            OTF_filename = os.path.split(otf)[1].split('.')[0]
            base_kwargs.update(dict(nthreads=1, gammaApo=gammaApo, suppressR=suppressR, wiener=wiener))
            sim_kwargs.update(base_kwargs)
            print("\nSIMRECON BASE KEYWORD ARGUMENTS:" + str(base_kwargs))
            user_text_process += '_w_' + str(wiener)  #+ '_ls_' + str(round(ls,3)) + 'gApo_'+str(gammaApo)+'_supR_'+str(suppressR)+
            if filter_tiles:
                user_text_process += '_tile_filter_'

            #create processed file output name
            sim_kwargs["output_file"] = sim_kwargs["input_file"].replace(".mrc",'_proc_' + OTF_filename + '_' +
                                    user_text_process + ".mrc")

            tile_process_name = sim_kwargs["input_file"].replace(".mrc",'_proc_' + OTF_filename + '_' +
             user_text_process +  '_tile' + str(tile_size) + "_pad" + str(tile_overlap) + ".mrc") #_tile16' + "_pad8" + ".mrc") 
# 
            if len(tile_process_name) > 255:
                print(tile_process_name)
                print("""\nfilename-directory too long, > 256 characters. solutions to fix this:
            1) Rename your file, 2) copy and paste file into higher directory location with less parent directories""")
                break           


            if tile_process_name not in done_already:
                 

                    print('\nOTF FILENAME: ' +str(OTF_filename) + '\n')

                    print("SIM RECON DATA TO SAVE: " + str(tile_process_name) + "\n" + '\n' + '\n')
                    #print(tile_limits)
                    
                    %time sim_output = split_process_recombine(sim_kwargs["input_file"], tile_size, tile_overlap, sim_kwargs, tile_limits=tile_limits)
                    # HAD TO EDIT simrecon_utils.py lines 1488 and 1490 and comment out mrc.close() as it was failing.
                    
                    
                    with open(sim_output[0].replace(".mrc", ".txt"), "w") as myfile:
                        myfile.write(str(sim_kwargs))
                        myfile.write("\n" + "-" * 80 + "\n")
                        myfile.write("\n".join(sim_output[1]))
                    done_already.add(sim_output[0])
                    
           
            else:
                    print('DATASET ALREADY PROCESSED: ' + str(tile_process_name) + '\n' + '\n' + '\n')



# #

Datasets alread reconstructed: 
F:\ORCA_Data\2024-12-03 - Schuyler LID 769\SIM imaging\LID 769 position 8 560nm - ROI 1 _20241204_105558
F:\ORCA_Data\2024-12-03 - Schuyler LID 769\SIM imaging\LID 769 position 8 560nm - ROI 1 _20241204_105558

RAW DATA TO BE PROCESSED, PROCESS No.<function step at 0x00000237CC6A2A60>: F:\ORCA_Data\2024-12-03 - Schuyler LID 769\SIM imaging\LID 769 position 8 560nm - ROI 1 _20241204_105558\560 nm 615 45 filter 5 phases 0.81 NA React_All Linear SIM_cam1_1.mrc


SIMRECON BASE KEYWORD ARGUMENTS:{'nphases': 5, 'ndirs': 3, 'angle0': 1.29, 'negDangle': True, 'na': 0.85, 'nimm': 1.0, 'zoomfact': 2.0, 'background': 100.0, 'wiener': 0.0005, 'fastSIM': True, 'otfRA': True, 'dampenOrder0': True, 'k0searchall': True, 'equalizez': True, 'preciseapo': True, 'gammaApo': 0.5, 'suppressR': 2.0, 'keeporder2': True, 'nthreads': 1}

OTF FILENAME: 560 201909_19-20_best

SIM RECON DATA TO SAVE: F:\ORCA_Data\2024-12-03 - Schuyler LID 769\SIM imaging\LID 769 position 8 560nm - R

  0%|          | 0/16 [00:00<?, ?it/s]

Splitting and saving data:   0%|          | 0/256 [00:00<?, ?it/s]

[########################################] | 100% Completed | 65.86 s


Reading back processed:   0%|          | 0/256 [00:00<?, ?it/s]

Recombining:   0%|          | 0/256 [00:00<?, ?it/s]

MRC Raw data file is still open
MRC temp data file is still open
MRC Raw data file is now closed
MRC temp data file is now closed
CPU times: total: 59.2 s
Wall time: 1min 14s
